In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

##module for handling analysis grid
import pyproj
from grid import Grid, Converter, regular_grid

##modules that get state_variable from model output
from models import topaz, nextsim

##utility functions to handle basic io for state variables
from assim_tools.basic_io import *

In [2]:
##analysis grid defined as a regular polar stere grid
proj = pyproj.Proj('+proj=stere +lat_0=90 +lon_0=-30 +lat_ts=60')
x, y = regular_grid(-2.5e6, 1.5e6, -1.5e6, 2.5e6, 3e3)
grid = Grid(proj, x, y)

In [3]:
##analysis time
t = datetime(2007, 1, 10)

In [4]:
##get the land mask for the analysis grid
path = '/cluster/work/users/yingyue/data/nextsim_ens'
filename = path+'/001/field_'+t.strftime('%Y%m%dT%H%M%SZ')+'.bin'

##native nextsim grid and its converter to analysis grid
grid1 = nextsim.get_grid(filename)
conv = Converter(grid1, grid)

##read a variable and convert to analysis grid
tmp = nextsim.read_data(filename, 'M_conc')

##land mask is where the converted variable is nan
mask = np.isnan(conv.convert(tmp))


In [5]:
##list of available state variable names
variables

{'atmos_surf_wind': {'is_vector': True, 'units': 'm/s'},
 'atmos_surf_temp': {'is_vector': False, 'units': 'K'},
 'atmos_surf_dew_temp': {'is_vector': False, 'units': 'K'},
 'atmos_surf_press': {'is_vector': False, 'units': 'Pa'},
 'atmos_precip': {'is_vector': False, 'units': 'kg/m2/s'},
 'atmos_snowfall': {'is_vector': False, 'units': 'kg/m2/s'},
 'atmos_down_shortwave': {'is_vector': False, 'units': 'W/m2'},
 'atmos_down_longwave': {'is_vector': False, 'units': 'W/m2'},
 'seaice_conc': {'is_vector': False, 'units': '%'},
 'seaice_thick': {'is_vector': False, 'units': 'm'},
 'snow_thick': {'is_vector': False, 'units': 'm'},
 'seaice_drift': {'is_vector': True, 'units': 'm'},
 'seaice_velocity': {'is_vector': True, 'units': 'm/s'},
 'seaice_damage': {'is_vector': False, 'units': '%'},
 'ocean_surf_temp': {'is_vector': False, 'units': 'K'},
 'ocean_surf_velocity': {'is_vector': True, 'units': 'm/s'},
 'ocean_surf_height': {'is_vector': False, 'units': 'm/s'},
 'ocean_temp': {'is_vector

In [ ]:
##create a field_info for the state variable
state_def_file = '/cluster/home/yingyue/code/NEDAS/config/state_def'

info = field_info(state_def_file,  ##state definition file, see config.__init__.py
                  t,               ##analysis time (datetime obj)
                  (-24, 0),        ##time window (left offset, right offset) in hours
                  np.arange(-50, 1),  ##vertical level indices (ocean goes -1, -2..., atmos goes +1, +2,...)
                  20,              ##ensemble size
                  *x.shape,        ##grid shape: ny, nx
                  mask             ##the land mask
                 )
info

In [6]:
##name of the state variable bin file
binfile = '/cluster/work/users/yingyue/prior.bin'

##we write "info" into a dat file with same basename as binfile
write_field_info(binfile, info)

In [7]:
##the first record in the binfile is the land mask,
##write_mask also creates the binfile if it doesn't exist
write_mask(binfile, info, mask)

In [ ]:
##processing nextsim variables
path = '/cluster/work/users/yingyue/data/nextsim_ens'

##dict to convert variables name to native nextsim names
var_dict = nextsim.var_dict

##loop over time slices
t_offsets = info['time_window']
var_dt = 24  ##should get from info
for n in np.arange(int(t_offsets[0]/var_dt), int(t_offsets[1]/var_dt)+1):
    tn = t + n*var_dt*timedelta(hours=1)

    ##loop over members
    for m in range(info['nens']):
        filename = path+'/{:03d}'.format(m+1)+'/field_'+tn.strftime('%Y%m%dT%H%M%SZ')+'.bin'
    
        ##each time and member will have different mesh configuration, need a separate converter
        grid1 = nextsim.get_grid(filename)
        conv = Converter(grid1, grid)
    
        ##loop over the variables in state
        for v in [info['var_names'][i] for i, s in enumerate(info['var_sources']) if s=='models.nextsim']:
            is_vector = variables[v]['is_vector']
            dat = nextsim.read_data(filename, var_dict[v]['name'])
            if is_vector:
                dat = dat.reshape((2, -1))
        
            ##convert to analysis grid
            dat_ = conv.convert(dat, is_vector=is_vector)
        
            ##write the dat_ field to the binfile
            print('writing variable '+v, ' member', m+1, 'time', tn)
            if is_vector:
                fid = list(set(i for i, rec in info['fields'].items() if rec['var_name']==v+'_x' and 
                               rec['member']==m and rec['time']==tn))[0]
                write_field(binfile, info, mask, fid, dat_[0,...])
                fid = list(set(i for i, rec in info['fields'].items() if rec['var_name']==v+'_y' and
                               rec['member']==m and rec['time']==tn))[0]
                write_field(binfile, info, mask, fid, dat_[1,...])
            else:
                fid = list(set(i for i, rec in info['fields'].items() if rec['var_name']==v and
                               rec['member']==m and rec['time']==tn))[0]
                write_field(binfile, info, mask, fid, dat_)
               

In [ ]:
##processing topaz variables
path = '/cluster/work/users/yingyue/data/TP4'
mask1 = topaz.get_mask(path+'/depth_TP4b0.12_01.a')

##hycom grid is fixed so one converter is enough
conv1 = Converter(topaz.grid, grid)

##let's pretend the TP4restart file tlevel=1,2 is this two time (for now)
for n, tn in enumerate((datetime(2007,1,9), datetime(2007,1,10))):
    for m in range(info['nens']):
        filename = path+'/TP4restart2007_002_00_mem{:03d}.a'.format(m+1)
        for z in range(50):  ##hycom variables have 50 vertical layers
            ##read ocean_velocity
            u  = topaz.read_data(filename, 'u', level=z+1, tlevel=n+1, mask=mask1)
            v  = topaz.read_data(filename, 'v', level=z+1, tlevel=n+1, mask=mask1)

            ##convert to analysis grid
            dat = conv1.convert(np.array([u, v]), is_vector=True)
            
            print('writing variable ocean_velocity', ' level', -1-z, ' member', m+1, 'time', tn)
            fid = list(set(i for i, rec in info['fields'].items() if rec['var_name']=='ocean_velocity_x' and
                           rec['member']==m and rec['level']==-1-z and rec['time']==tn))[0]
            write_field(binfile, info, mask, fid, dat[0,...])
            fid = list(set(i for i, rec in info['fields'].items() if rec['var_name']=='ocean_velocity_y' and
                           rec['member']==m and rec['level']==-1-z and rec['time']==tn))[0]
            write_field(binfile, info, mask, fid, dat[1,...])
